<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/matching/simlarity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install playwright
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [2]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument,Word2Vec
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import re
import plotly.graph_objects as go
import nltk
nltk.download('all', download_dir='/usr/local/lib/nltk_data')
from nltk import pos_tag
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]  

In [3]:
job_details_file = "job_details.txt"
profile_data_file = "profile_data.txt"


In [4]:
def extract_section(file_path, start_keyword, end_keyword):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    extracting = False
    extracted_text = []

    for line in lines:
        if start_keyword in line:
            extracting = True
        if extracting:
            extracted_text.append(line.strip())
        if end_keyword in line:
            break

    return "\n".join(extracted_text)

unique_skills_raw = extract_section(profile_data_file, "Unique skills:", "Unique skills:")

if unique_skills_raw.startswith("Unique skills:"):
    unique_skills = unique_skills_raw.replace("Unique skills:", "").strip()

# Convert skills to a formatted string
profile_skills = f"""['{unique_skills.replace(', ', "', '")}']"""



In [5]:
def extract_description(file_path, start_keyword, end_keyword=None):
    """
    Extract a specific section of text from a file based on start and optional end keywords.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    extracting = False
    extracted_text = []

    for line in lines:
        if start_keyword in line:
            extracting = True
        if extracting:
            extracted_text.append(line.rstrip())  # Keep formatting with spaces
        if end_keyword and end_keyword in line:
            break

    return "\n".join(extracted_text)

# File path to the uploaded text file
file_path = "job_details.txt"

# Extract the description section
job_description = extract_description(file_path, "Job description", "Description:")



In [6]:
# Extended stopwords for job descriptions
stop_words = set(stopwords.words('english')).union({
    'have', 'has', 'you', 'we', 'they', 'our', 'your', 'the', 'a', 'an', 'in', 'on', 'at', 'for', 'with', 'to', 'of',
    'and', 'or', 'is', 'are', 'was', 'were', 'be', 'will', 'can', 'should', 'would', 'could', 'may', 'might',
    'solid', 'relevant', 'extensive', 'proven', 'experience', 'possess', 'ability', 'skills', 'capabilities',
    'demonstrated', 'strong', 'knowledge', 'understanding', 'background', 'proficiency', 'required',
    'responsibilities', 'qualifications', 'advantage', 'must', 'preferred', 'environment', 'excellent', 'good',
    'key', 'various', 'well', 'including', 'new', 'opportunities', 'job', 'role', 'position'
})

# 1. Preprocessing Functions
def preprocess_text(text):
    """
    Preprocess text by removing excessive newlines, extra spaces, and special characters.
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    text = re.sub(r'\s*\n\s*', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\s+', ' ', text)       # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)    # Remove special characters
    return text.strip().lower()            # Lowercase and strip whitespace

def extract_relevant_phrases(sentence):
    """
    Extract relevant phrases from a sentence by removing unnecessary words and focusing on key phrases.
    """
    # Remove unnecessary phrases using regex
    cleaned_sentence = re.sub(r'\b(have|has|possess|proven|solid|relevant|extensive|demonstrated|strong)\b', '', sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(experience (in|with|on|of|for))\b', '', cleaned_sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(ability to|proficiency in|knowledge of|understanding of|background in)\b', '', cleaned_sentence, flags=re.IGNORECASE)

    # Tokenize and remove stopwords
    tokens = word_tokenize(cleaned_sentence.lower())
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]

    # Extract nouns using POS tagging
    pos_tags = pos_tag(tokens)
    relevant_phrases = [word for word, tag in pos_tags if tag.startswith('NN')]  # Keep only nouns

    return ' '.join(relevant_phrases)

def process_job_description(description):
    """
    Process job description to split into sentences and extract relevant parts.
    """
    # Split the description into sentences using punctuation or bullet points
    sentences = re.split(r'[.\n•;]', description)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]  # Remove empty strings

    # Further split sentences into manageable phrases if too long
    processed_sentences = []
    for sentence in sentences:
        # Split sentences longer than a threshold into smaller parts by words
        if len(sentence.split()) > 20:  # Threshold: 20 words
            chunks = re.split(r'(?:,| and | or )', sentence)  # Split by commas, "and", or "or"
            processed_sentences.extend([chunk.strip() for chunk in chunks if chunk.strip()])
        else:
            processed_sentences.append(sentence)

    # Extract relevant phrases from each chunk
    processed_chunks = [extract_relevant_phrases(preprocess_text(chunk)) for chunk in processed_sentences]

    # Remove empty results
    return [chunk for chunk in processed_chunks if chunk]

# 2. Main Workflow
def compute_similarity(job_description, profile_skills, model, threshold=0.5):
    """
    Compute similarity between job description and profile skills using embeddings.
    """
    # Preprocess inputs
    job_skills_processed = process_job_description(preprocess_text(job_description))
    profile_skills_list = [preprocess_text(skill) for skill in eval(profile_skills)]  # Convert and preprocess profile skills

    # Encode skills using Sentence-BERT
    job_embeddings = model.encode(job_skills_processed, convert_to_tensor=True)
    profile_embeddings = model.encode(profile_skills_list, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_similarities = util.pytorch_cos_sim(job_embeddings, profile_embeddings)

    # Adjust similarities based on threshold
    adjusted_similarities = []
    total_similarity = 0
    comparison_count = 0

    for i in range(len(job_skills_processed)):
        similarity_row = cosine_similarities[i].tolist()
        max_similarity = max(similarity_row)
        if max_similarity > threshold:
            similarity_row = [1.0] * len(similarity_row)
        adjusted_similarities.append(similarity_row)
        total_similarity += sum(similarity_row)
        comparison_count += len(similarity_row)

    regulezation_factor = 0.3
    # Calculate overall similarity
    overall_similarity = min(total_similarity / comparison_count + regulezation_factor, 1.0) if comparison_count > 0 else 0

    return adjusted_similarities, overall_similarity, job_skills_processed, profile_skills_list


In [7]:

model = SentenceTransformer('all-MiniLM-L6-v2')



# Compute similarities
adjusted_similarities, overall_similarity, job_skills_processed, profile_skills_processed = compute_similarity(
    job_description, profile_skills, model
)

# Display results
print("Adjusted Similarities:")
for i, job_skill in enumerate(job_skills_processed):
    for j, profile_skill in enumerate(profile_skills_processed):
        similarity_score = adjusted_similarities[i][j]
        print(f"Similarity between job skill '{job_skill}' and profile skill '{profile_skill}': {similarity_score:.2f}")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Adjusted Similarities:
Similarity between job skill 'description algorithm group' and profile skill 'analytical skills': 0.14
Similarity between job skill 'description algorithm group' and profile skill 'candidate management': 0.16
Similarity between job skill 'description algorithm group' and profile skill 'cosourcing': 0.11
Similarity between job skill 'description algorithm group' and profile skill 'communication': 0.23
Similarity between job skill 'description algorithm group' and profile skill 'corporate recruiting': 0.17
Similarity between job skill 'description algorithm group' and profile skill 'curriculum vitae cv': -0.02
Similarity between job skill 'description algorithm group' and profile skill 'english': 0.10
Similarity between job skill 'description algorithm group' and profile skill 'executive search': 0.18
Similarity between job skill 'description algorithm group' and profile skill 'game theory': 0.12
Similarity between job skill 'description algorithm group' and profil

In [8]:
print(f"\nOverall Similarity: {overall_similarity:.2f}")


Overall Similarity: 0.62


In [9]:
# Assume similarity is your final score as a percentage
similarity = overall_similarity * 100  # Convert to percentage if in range [0, 1]

# Visualization
fig = go.Figure(go.Indicator(
    domain={'x': [0, 1], 'y': [0, 1]},
    value=similarity,
    mode="gauge+number",
    title={'text': "Matching percentage (%)"},
    gauge={
        'axis': {'range': [0, 100]},
        'steps': [
            {'range': [0, 50], 'color': "#FFB6C1"},  # Light pink for low match
            {'range': [50, 70], 'color': "#FFFFE0"},  # Light yellow for moderate match
            {'range': [70, 100], 'color': "#90EE90"}  # Light green for high match
        ],
        'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': similarity}  # Mark the current similarity
    }
))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()
